### SNR requirement

The SNR analysis is based on the following idea:
1. We use the instrument to derive a realistic SNR and L2 estimate. It does not make much sense to work with a simplified SNR model like in the siml1b module as it is important to take here already the choice of the detector into acount. With the InGaAs specific QE, the SNR as a specific spectral shape, that need to be accounted here. 
2. We simulated the reference scene and scale it with a factor to vary SNR 
3. This allows us to determine the mean SNR over the two fit windows 1597.0-1617.0 nm and 1664.4-1668.0 nm  and the L2 precsiion as a function of the scaling factor.
4. Having a requirement on the L2 precision of 3.8 ppm for the reference scene, we can derive the scaling factor, and so the required mean SNR for the fit windows. 
5. The resilence of the approach with respect to dead and bad pixels will be tested. 

#### 1. Set working enviroment

In [1]:
import sys
import os
import yaml
import numpy as np
from netCDF4 import Dataset
import matplotlib
# define  path to search for module
sys.path.append("/home/jochen/TANGO_E2ES/EndtoEndProject/end_to_end/")
os.chdir("/home/jochen/TANGO_E2ES/EndtoEndProject/end_to_end/examples/exp5_snr/")

#### 2. import teds modules

In [2]:
from teds.gm import geometry_module
from teds.sgm import geoscene_generation
from teds.sgm import Carbon_radiation_scene_generation
from teds.im import run_instrument_model
from teds.l1al1b import run_l1al1b
from teds.l1l2.l1bl2 import level1b_to_level2_processor_RTorCH4, level1b_to_level2_processor
from teds.lib.libNumTools import get_isrf

#### 3. Operational flags

In [3]:
proc_gm     = False
proc_sgmgeo = False
proc_sgmrad = False
proc_im     = True
proc_l1al1b = True
proc_l1bl2  = True

data_alys   = True 

Set variables applicable to all cells

In [4]:
path  = '/home/jochen/TANGO_E2ES/EndtoEndProject/data/interface_data/'
nsca  = 40
scale = np.arange(nsca)*0.05 + 0.05 #radiance scaling 
nsca  = 1
scale = np.arange(nsca) + 1. #radiance scaling 


print(scale)

[1.]


In [5]:
if(proc_gm):
    #configureation file
    #In this case, the gm does not include solar and viewing geometry. It is easier to define them with the notebook
    gm_config= yaml.safe_load(open('./settings/gm_config.yaml'))
    #we use the nact dimension to simulate scenes with different scaling factors s and the geometry of Lref
    nact = 100 
    gm_config['scene_spec'] = {}
    gm_config['scene_spec']['sza'] = np.zeros(nact) + 70.
    gm_config['scene_spec']['saa'] = np.zeros(nact) 
    gm_config['scene_spec']['vza'] = np.zeros(nact) 
    gm_config['scene_spec']['vaa'] = np.zeros(nact) 

    geometry_module(gm_config)


#### 5. SGM-GEO

Here we make use of the fact that for a non-scattering RT simulation a scaling of the radiance spectrum is equivalent with a scaling of the albedo.

In [6]:
if(proc_sgmgeo):
    albedo = np.ones(nact)*0.15
    for sc in scale:
        sgmgeo_config= yaml.safe_load(open('./settings/sgmgeo_config.yaml'))
        sgmgeo_config['scene_spec']={}
        sgmgeo_config['scene_spec']['albedo'] = albedo*sc
        sgmgeo_config['io_files']['output_geo']=path+'sgm/Tango_Carbon_sgm_atmosphere_exp5.0_scale_'+("%.3f" % (sc)) + '.nc'
        geoscene_generation(sgmgeo_config)

#### 6. SGM-RAD

In [7]:
if(proc_sgmrad):

    for sc in scale:
        sgmrad_config= yaml.safe_load(open('./settings/sgmrad_config.yaml'))
        sgmrad_config['io_files']['input_sgm_geo']=path+'sgm/Tango_Carbon_sgm_atmosphere_exp5.0_scale_'+("%.3f" % (sc)) + '.nc'
        sgmrad_config['io_files']['output_rad']=path + 'sgm/Tango_Carbon_sgm_radiance_exp5.0_scale_'+("%.3f" % (sc)) + '.nc'
        Carbon_radiation_scene_generation(sgmrad_config)

### 7. Instrument model 

Run the instrument model for each scaled scenes using a seperate call 

In [8]:
if(proc_im):

    for sc in scale:
        im_config= yaml.safe_load(open('./settings/im_config.yaml'))
        im_config['io']['sgm'] = path + 'sgm/Tango_Carbon_sgm_radiance_exp5.0_scale_'+("%.3f" % (sc)) + '.nc'
        im_config['io']['l1a'] = path + 'level1a/Tango_Carbon_l1a_exp5.0_scale_'+("%.3f" % (sc)) + '.nc'
        run_instrument_model(im_config)

[20:00:35] Processing from SGM to l1a


##########################
# Tango instrument model #
##########################
Version                 : 0.0.1
Commit hash             : 56597a5b
Date and timezone       : 2024 December 27 Fri
Contacts                : raullaasner@gmail.com
                          bitbucket.org/sron_earth/teds/issues (request permission)
Host system             : Linux 6.8.0-50-generic

##############################
# Reading CKD and input data #
##############################


[20:00:36] Reading input data
[20:00:36] ISRF convolution



#################
# Forward model #
#################


100%|██████████| 1/1 [00:00<00:00,  4.00 ALT/s]
[20:00:37] Radiometric
100%|██████████| 1/1 [00:00<00:00, 7410.43it/s]
[20:00:37] Detector mapping
100%|██████████| 1/1 [00:34<00:00, 34.76s/it]
[20:01:12] Stray light
100%|██████████| 1/1 [00:00<00:00,  1.08it/s]
[20:01:13] PRNU
[20:01:13] Nonlinearity
[20:01:13] Dark signal
[20:01:13] Noise
[20:01:13] Dark offset
[20:01:13] Coadding and binning
[20:01:13] Writing output data



###########
# Success #
###########


#### 8. L1B processing

In [9]:
if(proc_l1al1b):
    for sc in scale:
        l1al1b_config= yaml.safe_load(open('./settings/l1al1b_config.yaml'))
        l1al1b_config['io']['l1a'] = path + 'level1a/Tango_Carbon_l1a_exp5.0_scale_'+("%.3f" % (sc)) + '.nc'
        l1al1b_config['io']['l1b'] = path + 'level1b/Tango_Carbon_l1b_exp5.0_scale_'+("%.3f" % (sc)) + '.nc'
        run_l1al1b(l1al1b_config)

[20:01:13] [WARNING] unrecognized input parameter: reverse_wavelength
[20:01:13] [WARNING] unrecognized input parameter: swath
[20:01:13] Processing from L1A to L1B


#######################
# Tango L1B processor #
#######################
Version                 : 0.0.1
Commit hash             : 56597a5b
Date and timezone       : 2024 December 27 Fri
Contacts                : raullaasner@gmail.com
                          bitbucket.org/sron_earth/teds/issues (request permission)
Host system             : Linux 6.8.0-50-generic

##############################
# Reading CKD and input data #
##############################


[20:01:14] Reading input data
[20:01:14] Coadding and binning
[20:01:14] Dark offset
[20:01:14] Noise
[20:01:15] Dark current
[20:01:15] Nonlinearity



#############
# Retrieval #
#############


100%|██████████| 1/1 [00:00<00:00, 41.64it/s]
[20:01:15] PRNU
100%|██████████| 1/1 [00:00<00:00, 69.31it/s]
[20:01:15] Smoothing over bad values
100%|██████████| 1/1 [00:00<00:00,  8.81it/s]
[20:01:15] Stray light
100%|██████████| 1/1 [00:02<00:00,  2.91s/it]
[20:01:18] Mapping from detector
100%|██████████| 1/1 [00:34<00:00, 34.36s/it]
[20:01:52] Interpolating from intermediate to main CKD wavelength grids
[20:01:52] Radiometric
[20:01:52] Writing output data



###########
# Success #
###########


#### 8. L2 processing of the $\mathrm{CO_2}$ and $\mathrm{CH_4}$ proxy product

* First reference simulation with gain simulations
* Loop over different isrf perturbations (acoeff and bcoeff of the generalized normal distribution)

In [10]:
if(proc_l1bl2):

    for sc in scale:
        l1bl2_config= yaml.safe_load(open('./settings/l1bl2_config.yaml'))
        l1bl2_config['io_files']['input_l1b'] = path + 'level1b/Tango_Carbon_l1b_exp5.0_scale_'+("%.3f" % (sc)) + '.nc'
        l1bl2_config['io_files']['output_l2'] = path + 'level2/Tango_Carbon_l2_exp5.0_scale_'+("%.3f" % (sc)) + '.nc'
        level1b_to_level2_processor(l1bl2_config)


level 1B to 2 proessor ...


100%|██████████| 1/1 [01:35<00:00, 95.71s/it]

/home/jochen/TANGO_E2ES/EndtoEndProject/data/interface_data/level2/Tango_Carbon_l2_exp5.0_scale_1.000.nc
=> l1bl2 finished successfully


#### 9. Data analysis: Sensitivity to fwhm (acoeff)

9.1 Read in data incld gains for reference case

In [11]:
if(data_alys):
    import netCDF4 as nc
    import numpy as np
    from copy import deepcopy
    import matplotlib.pyplot as plt
    import sys
    plt.rcParams.update({'font.size': 14,})
    path ='/home/jochen/TANGO_E2ES/EndtoEndProject/data/interface_data/'

    nact  = 100
    nwave = 550

    snr_max= np.zeros((nsca))
    snr_min= np.zeros((nsca))
    rad_max= np.zeros((nsca))
    rad_min= np.zeros((nsca))
    l2prec_median = np.zeros((nsca))
    co2 = np.zeros((nsca))

    for isca, sc in enumerate(scale):
        print(isca, sc)
        filenamel1b = path + 'level1b/Tango_Carbon_l1b_exp5.0_scale_'+("%.3f" % (sc)) + '.nc'
        filenamel2  = path + 'level2/Tango_Carbon_l2_exp5.0_scale_'+("%.3f" % (sc)) + '.nc'

        l1b_data = nc.Dataset(filenamel1b)
        l2_data  = nc.Dataset(filenamel2)

        wave  = np.array(deepcopy(l1b_data['observation_data']['wavelength'][:]))
        rad   = np.array(deepcopy(l1b_data['observation_data']['radiance'][:]))
        noise = np.array(deepcopy(l1b_data['observation_data']['radiance_stdev'][:]))
        snr = rad/noise

        lmax = 114
        lmin = 502
        snr_max[isca] = np.median(snr[0,:,lmax]) 
        snr_min[isca] = np.median(snr[0,:,lmin])
        rad_max[isca] = np.median(rad[0,:,lmax]) 
        rad_min[isca] = np.median(rad[0,:,lmin])
    
        prec_xco2_proxy = np.array(deepcopy(l2_data['precision XCO2 proxy'][:]).flatten())
        l2prec_median[isca] = np.median(prec_xco2_proxy)

    print(snr_max)
    print(l2prec_median)
    sys.exit()

    l2_req   = 3.8
    wave_max = wave[0,lmax] 
    wave_min = wave[0,lmin]
    req_snr_max = np.interp(l2_req, np.flip(l2prec_median[:]), np.flip(snr_max[:]))
    req_snr_min = np.interp(l2_req, np.flip(l2prec_median[:]), np.flip(snr_min[:]))
    req_rad_max = rad_max[14]
    req_rad_min = rad_min[14]

    sw_fig1 = False
    sw_fig2 = True

    if(sw_fig1):
        fig = plt.figure(figsize=(26, 7), dpi=100,)
        ax1 = fig.add_subplot(121)
        ax1.plot(scale,snr_max[:], color = 'green', label = 'SNR$_\mathrm{max}$')
        ax1.plot(scale,snr_min[:], color = 'blue', label = 'SNR$_\mathrm{min}$')
        ax1.set_xlabel('scaling factor $s$ [1]')
        ax1.set_ylabel('SNR [1]')
        plt.legend()
    
    #sys.exit()
#     req_scaling = np.interp(l2_req, np.flip(l2prec_median[:]), np.flip(scale))
#     print('required scaling factor:', req_scaling)
    
#     ax2 = fig.add_subplot(122)
#     ax2.plot(albs/0.15,l2prec_median[:, isza], color = 'grey')
#     ax2.set_xlabel('scaling factor $s$ [1]')
#     ax2.set_ylabel('XCO$_2^\mathrm{prox}$ [ppm]')
    
#     textstr = '\n'.join((
#         r'required L2 precsion =%.2f' % (l2_req, ) + ' ppm',
#         r'degradation factor $s =%.2f$' % (req_scaling, )))
#     props = dict(boxstyle='round', facecolor='grey', alpha=0.2)
#     ax2.text(0.15, 0.95, textstr, transform=ax2.transAxes, fontsize=14,
#             verticalalignment='top', bbox=props)
#     plt.savefig('plots/l2_precision.png',)
    
    if(sw_fig2):

        fig = plt.figure(figsize=(11, 7), dpi=100,)
    
#     textstr_max = '\n'.join((
#         r'$\lambda_\mathrm{max}=%.2f$' % (wave_max, ) + 'nm',
#         r'$I_\mathrm{max}=%.2E$' % (req_rad_max, ) + ' ph/(s sr m$^2$ nm)',
#         r'$SNR_\mathrm{max}=%.2f$' % (req_snr_max, )))
    
#     textstr_min = '\n'.join((
#         r'$\lambda_\mathrm{min}=%.2f$' % (wave_min, ) + 'nm',
#         r'$I_\mathrm{min}=%.2E$' % (req_rad_min, ) + ' ph/(s sr m$^2$ nm)',
#         r'$SNR_\mathrm{min}=%.2f$' % (req_snr_min, )))
    
        ax1 = fig.add_subplot(111)
        ax1.plot(l2prec_median,snr_max, color = 'green', label = 'SNR$_\mathrm{max}$')
        ax1.plot(l2prec_median,snr_min, color = 'blue', label = 'SNR$_\mathrm{min}$')
        ax1.set_xlabel('XCO$_2^\mathrm{prox}$ [ppm]')
        ax1.set_ylabel('SNR [1]')
        plt.legend()
#     props = dict(boxstyle='round', facecolor='green', alpha=0.2)
#     ax1.text(0.15, 0.95, textstr_max, transform=ax1.transAxes, fontsize=14,
#             verticalalignment='top', bbox=props)
#     props = dict(boxstyle='round', facecolor='blue', alpha=0.2)
#     ax1.text(0.15, 0.65, textstr_min, transform=ax1.transAxes, fontsize=14,
#             verticalalignment='top', bbox=props)
#     plt.savefig('plots/snr.png',)
    
# plt.show()

0 1.0
[138.33611049]
[6.52001042]


SystemExit: 

/home/jochen/pythonenvs/tangoendtoend/bin/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


9.2 Plot the induced L2 error (fully iterated) for the fwhm perturbation (a-coefficient) as a function of SZA 

9.3 Plot the L2 induced error as a function of perturbed fwhm for the itertative solution and for linear error propagation.

In [ ]:
if(data_aly2_acoeff):

    nacoeff,nsza,nwave = radiance_per_acoeff.shape

    delta_XCO2ns    = np.zeros([nacoeff,nsza])
    delta_XCH4ns    = np.zeros([nacoeff,nsza])
    delta_XCO2      = np.zeros([nacoeff,nsza])

    for ia in range(nacoeff):
        for isza in range(nsza):
            deltarad = radiance_per_acoeff[ia,isza,:]-radiance_ref[isza,:]
            delta_XCO2ns[ia,isza] = np.dot(gain_co2_ns[0,isza,:],deltarad)/xco2_true[isza]*100.
            delta_XCH4ns[ia,isza] = np.dot(gain_ch4_ns[0,isza,:],deltarad)/xch4_true[isza]*100.
            delta_XCO2[ia, isza]  = np.dot(gain_co2_proxy[0,isza,:],deltarad)/xco2_true[isza]*100.

    fig = plt.figure(figsize=(22,6), dpi=100,)
    plt.rcParams.update({'font.size': 16})
    
    ax1 = fig.add_subplot(131)

    ax1.plot(acoeff, xco2_ns_error[:, 0], label='sza = '+"{:.2f}".format(sza[0]), color = 'green')
    ax1.plot(acoeff, xco2_ns_error[:, 3], label='sza = '+"{:.2f}".format(sza[3]), color = 'blue')
    ax1.plot(acoeff, xco2_ns_error[:, 7], label='sza = '+"{:.2f}".format(sza[7]), color = 'orange')

    ax1.plot(acoeff, delta_XCO2ns[:, 0], label='linear error prop.',linestyle = ':', color = 'green')
    ax1.plot(acoeff, delta_XCO2ns[:, 3], linestyle = ':', color = 'blue')
    ax1.plot(acoeff, delta_XCO2ns[:, 7], linestyle = ':', color = 'orange')

    ax1.set_xlabel('fwhm [nm]')
    ax1.set_ylabel('$\delta$XCO$_2$ [%]')
    ax1.legend(fontsize = '11')
    ax1.set_ylim([-0.5, 0.5])
    ax1.set_title('XCO$_2$ non-scattering')
    
    ax2 = fig.add_subplot(132)
    
    ax2.set_xlabel('fwhm [nm]')
    ax2.set_ylabel('$\delta$XCH$_4$ [%]')
    ax2.set_ylim([-5., 5.])
    ax2.set_title('XCH$_4$ non-scattering')

    ax2.plot(acoeff, xch4_ns_error[:, 0], label='sza = '+"{:.2f}".format(sza[0]), color = 'green')
    ax2.plot(acoeff, xch4_ns_error[:, 3], label='sza = '+"{:.2f}".format(sza[3]), color = 'blue')
    ax2.plot(acoeff, xch4_ns_error[:, 7], label='sza = '+"{:.2f}".format(sza[7]), color = 'orange')

    ax2.plot(acoeff, delta_XCH4ns[:, 0], linestyle = ':', color = 'green')
    ax2.plot(acoeff, delta_XCH4ns[:, 3], linestyle = ':', color = 'blue')
    ax2.plot(acoeff, delta_XCH4ns[:, 7], linestyle = ':', color = 'orange')
    ax3 = fig.add_subplot(133)
    
    ax3.set_xlabel('fwhm [nm]')
    ax3.set_ylabel('$\delta$XCO$_2$ [%]')
    ax3.set_ylim([-5., 5.])
    ax3.set_title('XCO$_2$ proxy')
    
    ax3.plot(acoeff, xco2_proxy_error[:, 0], label='sza = '+"{:.2f}".format(sza[0]), color = 'green')
    ax3.plot(acoeff, xco2_proxy_error[:, 3], label='sza = '+"{:.2f}".format(sza[3]), color = 'blue')
    ax3.plot(acoeff, xco2_proxy_error[:, 7], label='sza = '+"{:.2f}".format(sza[7]), color = 'orange')

    ax3.plot(acoeff, delta_XCO2[:, 0], label='sza = '+"{:.2f}".format(sza[0]),linestyle = ':', color = 'green')
    ax3.plot(acoeff, delta_XCO2[:, 3], label='sza = '+"{:.2f}".format(sza[3]),linestyle = ':', color = 'blue')
    ax3.plot(acoeff, delta_XCO2[:, 7], label='sza = '+"{:.2f}".format(sza[7]),linestyle = ':', color = 'orange')

    plt.savefig('plots/exp6_isrf_fwhm.png',)

#### 10. Data analysis: Sensitivity to fwhm (bcoeff)

10.1 Read in data incld gains for reference case

In [12]:
if(data_read_bcoeff):

    path = '/home/jochen/TANGO_E2ES/EndtoEndProject/data/interface_data/'
    za = [70., 60, 50, 40, 30, 20, 10, 0]
    nsza = len(sza)

    nbcoeff = 11
    bcoeff = np.arange(0, nbcoeff)*0.01 + 0.4
                    
    xco2_proxy = np.zeros([nbcoeff, nsza])
    xch4_proxy = np.zeros([nbcoeff, nsza])
    xco2_ns    = np.zeros([nbcoeff, nsza])
    xch4_ns    = np.zeros([nbcoeff, nsza])

    precision_xco2_proxy = np.zeros([nbcoeff, nsza])
    precision_xco2_ns    = np.zeros([nbcoeff, nsza])
    precision_xch4_ns    = np.zeros([nbcoeff, nsza])

    for ib in range(nbcoeff):    

        str_bcoeff = "%.3f" % (bcoeff[ib])
        filename = 'level2/Tango_Carbon_l2_exp6.0_bcoeff'+str_bcoeff+'.nc'
        l2_data = nc.Dataset(path+filename)

        xco2_proxy[ib, :] = deepcopy(l2_data['XCO2 proxy'][:])
        xch4_proxy[ib, :] = deepcopy(l2_data['XCH4 proxy'][:])

        xco2_ns[ib,:]    = deepcopy(l2_data['non_scattering_retrieval']['XCO2'][:]).flatten()
        xch4_ns[ib,:]    = deepcopy(l2_data['non_scattering_retrieval']['XCH4'][:]).flatten()
        prec_xco2_proxy[ib,:] = deepcopy(l2_data['precision XCO2 proxy'][:]).flatten()
        prec_xco2_ns[ib,:]    = deepcopy(l2_data['non_scattering_retrieval']['precision XCO2'][:]).flatten()
        prec_xch4_ns[ib,:]    = deepcopy(l2_data['non_scattering_retrieval']['precision XCH4'][:]).flatten()
        l2_data.close()

    filename = 'sgm/Tango_Carbon_sgm_atmosphere_exp6.0.nc'
    sgm_data = nc.Dataset(path+filename)
    xco2_true = deepcopy(sgm_data['XCO2'][:]).flatten()
    xch4_true = deepcopy(sgm_data['XCH4'][:]).flatten()
    sgm_data.close()

    xco2_proxy_error = np.zeros([nbcoeff, nsza])
    xch4_proxy_error = np.zeros([nbcoeff, nsza])
    xco2_ns_error = np.zeros([nbcoeff, nsza])
    xch4_ns_error = np.zeros([nbcoeff, nsza])

    for ib in range(nbcoeff):
            xco2_proxy_error[ib, :] = (xco2_proxy[ib, :]-xco2_true[:])/xco2_true[:]*100.
            xch4_proxy_error[ib, :] = (xch4_proxy[ib, :]-xch4_true[:])/xch4_true[:]*100.
            xco2_ns_error[ib, :] = (xco2_ns[ib, :]-xco2_true[:])/xco2_true[:]*100.
            xch4_ns_error[ib, :] = (xch4_ns[ib, :]-xch4_true[:])/xch4_true[:]*100.

    # read gains from diag files

    filel2_diag = '/home/jochen/TANGO_E2ES/EndtoEndProject/data/interface_data/level2/Tango_Carbon_l2_exp6.0_diag_acoeff0.450.nc'
    diag= Dataset(filel2_diag, mode='r')
    gain_ch4_ns    = diag['gain CH4'][:]
    gain_co2_ns    = diag['gain CO2'][:]
    
    # diag.variables.keys()

    nn, nsza, nwave = gain_co2_ns.shape
    radiance_ref    = np.zeros([nsza, nwave])
    radiance_ref[:,:] = diag['measurement'][:]
    wave_l1b        = diag['wavelength'][0,0,:]
    gain_co2_proxy  = np.zeros([nn,nsza,nwave])
    gain_ch4_proxy  = np.zeros([nn,nsza,nwave])

    for isza in range(nsza):
        gain_co2_proxy[0,isza,:] = (gain_co2_ns[0,isza,:]/xco2_ns[0,isza] - gain_ch4_ns[0,isza,:]/xch4_ns[0,isza])*xco2_proxy[0,isza]
        gain_ch4_proxy[0,isza,:] = (gain_ch4_ns[0,isza,:]/xch4_ns[0,isza] - gain_co2_ns[0,isza,:]/xco2_ns[0,isza])*xch4_proxy[0,isza]

    # simulated perturation of the measurement due to perturbed isrf, i.e., perturbation of the acoeff and bcoeff of 
    # the generalized normal distribution 
    filen_sgmrad = path + 'sgm/Tango_Carbon_sgm_radiance_exp6.0.nc'
    sgmrad = Dataset(filen_sgmrad)
    wave_lbl = sgmrad['wavelength'][:].data
    radiance_per_bcoeff = np.zeros([nbcoeff,nsza,nwave])

    isrf_config = {}
    isrf_config['type']   = 'generalized_normal' 
    isrf_config['fwhm'] = 0.45
    for ib in range(nbcoeff):
        isrf_config['bcoeff'] = 0.4 + ib*0.01
        isrf_convolution = get_isrf(wave_l1b, wave_lbl, isrf_config)
        for isza in range(nsza):
            radiance_per_bcoeff[ib, isza, :] = isrf_convolution(sgmrad['radiance'][0, isza, :].data)

10.2 Plot the induced L2 error (fully iterated) for the fwhm perturbation (b-coefficient) as a function of SZA

In [ ]:
if(data_aly1_bcoeff):
    fig = plt.figure(figsize=(22,8), dpi=100,)
    plt.rcParams.update({'font.size': 16})

    ax1 = fig.add_subplot(131)
    for ib in range(11):
        ax1.plot(sza, xco2_ns_error[ib, :], label='b = '+"{:.2f}".format(bcoeff[ib]))
    ax1.set_xlabel('SZA [degree]')
    ax1.set_ylabel('$\delta$XCO$_2$ [%]')
    ax1.legend(fontsize = '11', ncol = 3)
    ax1.set_ylim([-5., 5.])
    ax1.set_title('XCO$_2$ non-scattering')

    ax2 = fig.add_subplot(132)

    ax2.set_xlabel('SZA [degree]')
    ax2.set_ylabel('$\delta$XCH$_4$ [%]')
    ax2.set_ylim([-5., 5.])
    ax2.set_title('XCH$_4$ non-scattering')
    for ib in range(11):
        ax2.plot(sza, xch4_ns_error[ib, :], label='b = '+"{:.2f}".format(bcoeff[ib]))

    ax3 = fig.add_subplot(133)

    ax3.set_xlabel('SZA [degree]')
    ax3.set_ylabel('$\delta$XCO$_2$ [%]')
    ax3.set_ylim([-5., 5.])
    ax3.set_title('XCO$_2$ proxy')
    for ib in range(11):
        ax3.plot(sza, xco2_proxy_error[ib, :], label='b = '+"{:.2f}".format(bcoeff[ib]))
    plt.savefig('plots/exp6_isrf_bcoeff_sza.png',)

10.3 Plot the L2 induced error as a function of perturbed fwhm for the itertative solution and for linear error propagation.

In [ ]:
if(data_aly2_bcoeff):

    nbcoeff,nsza,nwave = radiance_per_bcoeff.shape

    delta_XCO2ns    = np.zeros([nbcoeff,nsza])
    delta_XCH4ns    = np.zeros([nbcoeff,nsza])
    delta_XCO2      = np.zeros([nbcoeff,nsza])

    for ib in range(nbcoeff):
        for isza in range(nsza):
            deltarad = radiance_per_bcoeff[ib,isza,:]-radiance_ref[isza,:]
            delta_XCO2ns[ib,isza] = np.dot(gain_co2_ns[0,isza,:],deltarad)/xco2_true[isza]*100.
            delta_XCH4ns[ib,isza] = np.dot(gain_ch4_ns[0,isza,:],deltarad)/xch4_true[isza]*100.
            delta_XCO2[ib, isza]  = np.dot(gain_co2_proxy[0,isza,:],deltarad)/xco2_true[isza]*100.

    fig = plt.figure(figsize=(22,6), dpi=100,)
    plt.rcParams.update({'font.size': 16})
    
    ax1 = fig.add_subplot(131)

    ax1.plot(bcoeff, xco2_ns_error[:, 0], label='sza = '+"{:.2f}".format(sza[0]), color = 'green')
    ax1.plot(bcoeff, xco2_ns_error[:, 3], label='sza = '+"{:.2f}".format(sza[3]), color = 'blue')
    ax1.plot(bcoeff, xco2_ns_error[:, 7], label='sza = '+"{:.2f}".format(sza[7]), color = 'orange')

    ax1.plot(bcoeff, delta_XCO2ns[:, 0], label='linear error prop.',linestyle = ':', color = 'green')
    ax1.plot(bcoeff, delta_XCO2ns[:, 3], linestyle = ':', color = 'blue')
    ax1.plot(bcoeff, delta_XCO2ns[:, 7], linestyle = ':', color = 'orange')

    ax1.set_xlabel('b [1]')
    ax1.set_ylabel('$\delta$XCO$_2$ [%]')
    ax1.legend(fontsize = '11')
    ax1.set_ylim([-0.5, 0.5])
    ax1.set_title('XCO$_2$ non-scattering')
    
    ax2 = fig.add_subplot(132)
    
    ax2.set_xlabel('b [1]')
    ax2.set_ylabel('$\delta$XCH$_4$ [%]')
    ax2.set_ylim([-5., 5.])
    ax2.set_title('XCH$_4$ non-scattering')

    ax2.plot(bcoeff, xch4_ns_error[:, 0], label='sza = '+"{:.2f}".format(sza[0]))
    ax2.plot(bcoeff, xch4_ns_error[:, 3], label='sza = '+"{:.2f}".format(sza[3]))
    ax2.plot(bcoeff, xch4_ns_error[:, 7], label='sza = '+"{:.2f}".format(sza[7]))

    ax2.plot(bcoeff, delta_XCH4ns[:, 0], linestyle = ':', color = 'green')
    ax2.plot(bcoeff, delta_XCH4ns[:, 3], linestyle = ':', color = 'blue')
    ax2.plot(bcoeff, delta_XCH4ns[:, 7], linestyle = ':', color = 'orange')
    ax3 = fig.add_subplot(133)
    
    ax3.set_xlabel('b [1]')
    ax3.set_ylabel('$\delta$XCO$_2$ [%]')
    ax3.set_ylim([-5., 5.])
    ax3.set_title('XCO$_2$ proxy')
    
    ax3.plot(bcoeff, xco2_proxy_error[:, 0], label='sza = '+"{:.2f}".format(sza[0]))
    ax3.plot(bcoeff, xco2_proxy_error[:, 3], label='sza = '+"{:.2f}".format(sza[3]))
    ax3.plot(bcoeff, xco2_proxy_error[:, 7], label='sza = '+"{:.2f}".format(sza[7]))

    ax3.plot(bcoeff, delta_XCO2[:, 0], label='sza = '+"{:.2f}".format(sza[0]),linestyle = ':', color = 'green')
    ax3.plot(bcoeff, delta_XCO2[:, 3], label='sza = '+"{:.2f}".format(sza[3]),linestyle = ':', color = 'blue')
    ax3.plot(bcoeff, delta_XCO2[:, 7], label='sza = '+"{:.2f}".format(sza[7]),linestyle = ':', color = 'orange')

    plt.savefig('plots/exp6_isrf_bcoeff.png',)

#### 11. Analysis L2 error as a function of the ISRF standard deviation
This is the basis for the TANGO Carbon ISRF requirement

In [15]:
# the generalized normal distribution
import math
def generalized_normal(x, fwhm, b=0.5, deriv=False):
    """Generalized normal distribution.

    :param x: A real value or a numpy array of real values.
    :param float fwhm: Full width at half-maximum.
    :param float b: Shape parameter. Value 0.5 for Gauss, towards 0 for a more
        blocky shape, towards 1 for stronger wings.
    :param bool deriv: If True, give the derivative instead of the
        distribution itself. Default is False.
    :return: Value(s) of generalized normal distribution (or its derivative if
        deriv=True) at `x`, normalized to sum 1 for kernels.
    :rtype: list(float)
    """
    const = np.log(2)**b/(fwhm*math.gamma(1+b))
    y = const*2**(-(2*np.abs(x)/fwhm)**(1/b))
    y /= np.sum(y)
    if deriv:
        y *= -(2/fwhm)**(1/b)*np.log(2)*np.sign(x)*np.abs(x)**(1/b-1)/b
    return y


In [ ]:
if(req_aly):

    path = '/home/jochen/TANGO_E2ES/EndtoEndProject/data/interface_data/'
    sza = [70., 60, 50, 40, 30, 20, 10, 0]
    nsza = len(sza)

    nacoeff = 15
    nbcoeff = 11
    bcoeff = np.arange(0, nbcoeff)*0.01 + 0.4
    acoeff = np.arange(0, nacoeff)*0.004 + 0.422
                    
    # The proxy product for perturbations of the acoeff and bcoeff of the ISRF
    bxco2_proxy = np.zeros([nbcoeff, nsza])
    bxch4_proxy = np.zeros([nbcoeff, nsza])
    axco2_proxy = np.zeros([nacoeff, nsza])
    axch4_proxy = np.zeros([nacoeff, nsza])

    # ISRF standard deviation
    bstd = np.zeros([nbcoeff, nsza])
    astd = np.zeros([nacoeff, nsza])
    # generate a wavelength grid for ISRF calculations centered around zero of a width +- 1 nm. 
    # This is needed to calculate the standard deviation of the ISRF 
    wave = (np.arange(2000)-1000)/1000
    nwave = wave.size

    # Get the proxy product from the L2 files
    for ib in range(nbcoeff):    
        acoeff_ref = 0.45
        isrf = generalized_normal(wave, acoeff_ref, bcoeff[ib])
        bstd[ib,:] = np.sqrt(np.power(wave, 2).dot(isrf))
        str_bcoeff = "%.3f" % (bcoeff[ib])
        filename = 'level2/Tango_Carbon_l2_exp6.0_bcoeff'+str_bcoeff+'.nc'
        dataset = nc.Dataset(path+filename)
        bxco2_proxy[ib, :] = deepcopy(dataset['XCO2 proxy'][:])
        bxch4_proxy[ib, :] = deepcopy(dataset['XCH4 proxy'][:])
        dataset.close()

    for ia in range(nacoeff):    
        bconst_fix = 0.45
        isrf = generalized_normal(wave, acoeff[ia], bconst_fix)
        astd[ia,:] = np.sqrt(np.power(wave, 2).dot(isrf))
        str_acoeff = "%.3f" % (acoeff[ia])
        filename = 'level2/Tango_Carbon_l2_exp6.0_acoeff'+str_acoeff+'.nc'
        dataset = nc.Dataset(path+filename)
        axco2_proxy[ia, :] = deepcopy(dataset['XCO2 proxy'][:])
        axch4_proxy[ia, :] = deepcopy(dataset['XCH4 proxy'][:])
        dataset.close()

    # Get the ground truth from the SGM
    filename = 'sgm/Tango_Carbon_sgm_atmosphere_exp6.0.nc'
    sgm_data = nc.Dataset(path+filename)
    xco2_true = deepcopy(sgm_data['XCO2'][:]).flatten()
    xch4_true = deepcopy(sgm_data['XCH4'][:]).flatten()
    sgm_data.close()

    #proxy error
    axco2_proxy_error = np.zeros([nacoeff, nsza])
    axch4_proxy_error = np.zeros([nacoeff, nsza])
    bxco2_proxy_error = np.zeros([nbcoeff, nsza])
    bxch4_proxy_error = np.zeros([nbcoeff, nsza])

    for ia in range(nacoeff):
            axco2_proxy_error[ia, :] = (axco2_proxy[ia, :]-xco2_true[:])/xco2_true[:]*100.
            axch4_proxy_error[ia, :] = (axch4_proxy[ia, :]-xch4_true[:])/xch4_true[:]*100.

    for ib in range(nbcoeff):
            bxco2_proxy_error[ib, :] = (bxco2_proxy[ib, :]-xco2_true[:])/xco2_true[:]*100.
            bxch4_proxy_error[ib, :] = (bxch4_proxy[ib, :]-xch4_true[:])/xch4_true[:]*100.

    std_ref = bstd[5,0]
    plt.scatter(bstd.flatten()-std_ref,bxco2_proxy_error.flatten(),label='$b_\mathrm{coeff}$', color = 'blue')
    plt.scatter(astd.flatten()-std_ref,axco2_proxy_error.flatten(),label='$a_\mathrm{coeff}$', color = 'green')

    #linear regression through all data points
    std = np.concatenate((astd.flatten(), bstd.flatten(),))
    xco2_error = np.concatenate((axco2_proxy_error.flatten(),bxco2_proxy_error.flatten()))
    std = std - bstd[5,0]
    A = np.vstack([std, np.ones(len(std))]).T
    A = std[:,np.newaxis]
    m, _, _, _ = np.linalg.lstsq(A, xco2_error)
    plt.plot(std, m*std, 'r', label='line regression', color='grey')

    #assume an CO2 proxy error of 0.25 %. Calculate the correspsonding std dev error that is acceptable
    e_max = (0.25)/m 
    plt.text(-0.004,-2.2,'$\delta\sigma(\pm 0.25\%) = \pm $'+"%.2E" % (e_max) + ' nm')
    plt.legend()
    plt.xlabel('$\delta \sigma$ [nm]')
    plt.ylabel('$\delta$ XCO$_2$ [%]')
    plt.savefig('plots/exp6_isrf_std_dev.png',)